In [75]:
%pylab inline
%load_ext line_profiler

import os
import pandas as pd
from pyvirchow.deep_model.model import get_model
#, predict_on_batch, predict_on_patch, slide_level_map
from pyvirchow.io.operations import WSIReader

from skimage.io import imread

from pyvirchow.morphology.mask import get_common_interior_polygons
from pyvirchow.io.operations import get_annotation_polygons
from pyvirchow.io.operations import poly2mask
from pyvirchow.io.operations import translate_and_scale_polygon
from pyvirchow.io.operations import WSIReader

from skimage.filters import threshold_otsu
from openslide.deepzoom import DeepZoomGenerator

from shapely.geometry import Point as shapelyPoint
from shapely.geometry import Polygon as shapelyPolygon

from tqdm import tqdm
import joblib

def get_tiles_fast(batch_samples,
                   patch_size=256,
                   num_classes=2,
                   convert_to_cat=True):
    X_train = batch_samples.img_path.apply(lambda x: joblib.load(x)).tolist()
    X_train = np.array(X_train)
    y_train = batch_samples.mask_path.apply(lambda x: joblib.load(x)).tolist()
    return X_train, y_train

def mask_to_categorical(y, num_classes=2, patch_size=256):
    """Convert binary mask to categorical array for Keras/TF.

    Parameters
    ----------
    y: array
       Input array
    num_classes: int
                 Number of classes (2)
    patch_size: int
                Original patch size

    Output
    ------
    y_cat: array
           Output array
    """
    from keras.utils.np_utils import to_categorical
    y_cat = to_categorical(
        y, num_classes=num_classes).reshape(y.shape[0], patch_size, patch_size,
                                            num_classes)
    return y_cat


def create_tumor_mask_from_tile(tile_x, tile_y, polygons, patch_size=256):
    """Create a patch_size x patch_size mask from tile_x,y coordinates
    Parameters
    ----------
    tile_x, tile_y:  int
    polygons: dict
              ['normal', 'tumor'] with corresponding polygons

    Returns
    -------
    mask: array
          patch_size x patch_size binary mask

    """

    # Initiate a zero mask
    mask = np.zeros((patch_size, patch_size))
    #patch_polygon = shapelyRectangle(tile_x, tile_y, patch_size, patch_size)
    x_min = tile_x
    y_min = tile_y
    x_max = x_min + 256
    y_max = y_min + 256
    patch_polygon = shapelyPolygon([(x_min, y_min), (x_max, y_min),
                                    (x_max, y_max), (x_min, y_max)])

    # Is it overlapping any of the tumor polygons?
    is_inside_tumor = [
        patch_polygon.intersection(polygon.buffer(0))
        for polygon in polygons['tumor']
    ]

    # the patch will always be inside just one annotated boundary
    # which are assumed to be non-overlapping and hence we can just fetch
    # the first sample
    tumor_poly_index = None
    tumor_poly_coords = None
    for index, sample_intersection in enumerate(is_inside_tumor):
        if sample_intersection.area > 0:
            tumor_poly_index = index
            if sample_intersection.geom_type == 'Polygon':
                tumor_poly_coords = np.array(
                    sample_intersection.exterior.coords)
            elif sample_intersection.geom_type == 'MultiPolygon':
                tumor_poly_coords = []
                for p in sample_intersection:
                    tumor_poly_coords += p.exterior.coords
                tumor_poly_coords = np.array(tumor_poly_coords)
            elif sample_intersection.geom_type == 'GeometryCollection':
                tumor_poly_coords = []
                for p in sample_intersection:
                    if p.geom_type == 'LineString' or p.geom_type == 'Point':
                        tumor_poly_coords += p.coords
                    elif p.geom_type == 'Polygon':
                        tumor_poly_coords += p.exterior.coords
                    else:
                        print('Found geom_type:{}'.format(p.geom_type))
                        raise ValueError('')
            else:
                print('Found geom_type:{}'.format(
                    sample_intersection.geom_type))
                raise ValueError('')
            break

    if tumor_poly_index is None:
        # No overlap with tumor so must return as is
        return mask

    # This path belongs to a tumor patch so set everything to one
    # Set these coordinates to one

    # Shift the tumor coordinates to tile_x, tile_y
    tumor_poly_coords = tumor_poly_coords - np.array([tile_x, tile_y])
    overlapping_tumor_poly = shapelyPolygon(tumor_poly_coords)
    # Create a psuedo mask
    psuedo_mask = poly2mask([overlapping_tumor_poly], (patch_size, patch_size))
    # Add it to the original mask
    mask = np.logical_or(mask, psuedo_mask)

    # If its inside tumor does this tumor patch actually contain any normal patches?
    tumor_poly = polygons['tumor'][tumor_poly_index]
    normal_patches_inside_tumor = get_common_interior_polygons(
        tumor_poly, polygons['normal'])

    # For all the normal patches, ensure
    # we set the mask to zero
    for index in normal_patches_inside_tumor:
        normal_poly = polygons['normal'][index]

        # What is the intersection portion of this normal polygon
        # with our patch of interest?
        common_area = normal_poly.intersection(patch_polygon)
        if not common_area.is_valid:
            return mask
        if common_area.geom_type == 'Polygon':
            normal_poly_coords = np.array(
                common_area.exterior.coords) - np.array([tile_x, tile_y])
        elif common_area.geom_type == 'MultiPolygon':
            normal_poly_coords = []
            for p in common_area:
                normal_poly_coords += p.exterior.coords
            normal_poly_coords = np.array(normal_poly_coords) - np.array(
                [tile_x, tile_y])
        if common_area:
            overlapping_normal_poly = shapelyPolygon(normal_poly_coords)
            psuedo_mask = poly2mask([overlapping_normal_poly],
                                    (patch_size, patch_size))
            # Get coordinates wherever this is non zero
            non_zero_coords = np.where(psuedo_mask > 0)
            # Add set these explicitly to zero
            mask[non_zero_coords] = 0
    return mask

def get_approx_tumor_mask(polygons,
                          thumbnail_nrow,
                          thumbnail_ncol,
                          patch_size=256):
    """Indicate whether a particular tile overlaps with tumor annotation.

    The method has an approx in its name, as the entire tile
    might not be coming from a tumor annotated region as parts of
    it might actually be normal. We just report
    here if the patch overlaps with a tumor annotation. It might have
    an overlap with a normal region as well, but that is filtered
    in a later method so we don't worry about it here.

    Parameters
    ----------
    polygons: dict
              dict with keys ['normal', 'tumor']
              as obtained from `get_annotation_polygons`
    thumbnail_nrow: int
                    Number of rows in the thumbnail image
    thumbnail_ncol: int
                    Number of columns in the thumbnail

    Returns
    -------
    mask: array
          tumor mask
    """
    scaled_tumor_polygons = []
    for tpol in polygons['tumor']:
        scaled = translate_and_scale_polygon(tpol, 0, 0, 1 / 256)
        scaled_tumor_polygons.append(scaled)
    polymasked = poly2mask(scaled_tumor_polygons,
                           (thumbnail_nrow, thumbnail_ncol))
    # Is any of the masked out points inside a normal annotated region?
    poly_x, poly_y = np.where(polymasked > 0)
    set_to_zero = []
    for px, py in zip(poly_x, poly_y):
        point = shapelyPoint(px, py)
        for npol in polygons['normal']:
            scaled = translate_and_scale_polygon(npol, 0, 0, 1 / 256)
            pol = shapelyPolygon(scaled.get_xy())
            if pol.contains(point):
                set_to_zero.append((px, py))

    if len(set_to_zero):
        set_to_zero = np.array(set_to_zero)
        polymasked[set_to_zero] = 0
    return polymasked

def get_all_patches_from_slide(slide_path,
                               json_filepath=None,
                               filter_non_tissue=True,
                               patch_size=256,
                               saveto=None):
    """Extract a dataframe of all patches

    Parameters
    ----------
    slide_path: string
                Path to slide
    json_filepath: string
                   Path to annotation file
    filter_non_tissue: bool
                       Should remove tiles with no tissue?
    saveto: string
            Path to store output

    Returns
    -------
    tiles_df: pd.DataFrame
              A dataframe with the following columns:
                  slide_path - abs. path to slide
                  uid - slide filename
                  is_tissue - True/False
                  is_tumor - True/False
                  tile_loc - (row, col) coordinate of the tile

    """
    with WSIReader(slide_path, 40) as slide:
        thumbnail = slide.get_thumbnail(
            (int(slide.dimensions[0] / patch_size),
             int(slide.dimensions[1] / patch_size)))
        thumbnail_nrow = int(slide.width / patch_size)
        thumbnail_ncol = int(slide.height / patch_size)

    thumbnail_grey = np.array(thumbnail.convert('L'))  # convert to grayscale

    thresh = threshold_otsu(thumbnail_grey)
    binary = thumbnail_grey > thresh

    patches = pd.DataFrame(pd.DataFrame(binary).stack())
    patches.loc[:, 'is_tissue'] = ~patches[0]
    patches.drop(0, axis=1, inplace=True)

    if json_filepath is not None:
        polygons = get_annotation_polygons(json_filepath)
        polymasked = get_approx_tumor_mask(polygons, thumbnail_nrow,
                                           thumbnail_ncol)

        patches_tumor = pd.DataFrame(pd.DataFrame(polymasked).stack())
        patches_tumor['is_tumor'] = patches_tumor[0] > 0
        patches_tumor.drop(0, axis=1, inplace=True)

        patches = pd.concat([patches, patches_tumor], axis=1)

    patches.loc[:, 'uid'] = os.path.basename(slide_path).replace('.tif', '')
    patches.loc[:, 'slide_path'] = os.path.abspath(slide_path)
    patches.loc[:, 'json_filepath'] = json_filepath
    if filter_non_tissue:
        patches = patches[patches.is_tissue ==
                          True]  # remove patches with no tissue
    patches['tile_loc'] = list(patches.index)
    patches.reset_index(inplace=True, drop=True)
    if saveto:
        patches.to_csv(saveto, sep='\t', index=False, header=True)
    return patches

def predict_on_batch(patches, model):
    """Predict pixel level probabilites of being a tumor of collection of patches.

    Parameters
    ----------
    patches: array_like
             A batch (list) of patches
    model: Keras model
           as obtained from get_model

    Returns
    -------
    prediction: array_like
                patch_size x patch_size x 1  per-pixel tumor probability
    """
    prediction = model.predict(patches)
    prediction = prediction[:, :, :, 1]
    return prediction


def process_batch(args):
    idx, model, batch_samples, batch_size = args
    output_thumbnail_pred = None
    X, _ = get_tiles(batch_samples)
    if batch_samples.is_tissue.nunique(
    ) == 1 and batch_samples.iloc[0].is_tissue == False:
        # all patches in this row do not have tissue, skip them all
        output_thumbnail_pred = np.zeros(batch_size, dtype=np.float32)

    else:
        # make predictions
        preds = predict_on_batch(X, model)
        output_thumbnail_pred = preds.mean(axis=(1, 2))
    return idx, output_thumbnail_pred


def slide_level_map(model, slide_path, batch_size=32, json_filepath=None):
    all_samples = get_all_patches_from_slide(slide_path, json_filepath, False,
                                             256)
    all_samples = all_samples.sample(frac=0.01)
    n_samples = len(all_samples.index)
    all_batch_samples = []
    for idx, offset in enumerate(list(range(0, n_samples, batch_size))):
        all_batch_samples.append((idx , model, all_samples.iloc[offset:offset + batch_size], batch_size))
    output_thumbnail_preds = []
    output_thumbnail_idx = []
    total = len(all_batch_samples)
    idx, o = process_batch(all_batch_samples[0])
    #print(idx)
    #print(o)
    #with Pool(processes=32) as p:
    #    with tqdm(total=total) as pbar:
    #for idx, result in (process_batch, all_batch_samples):
    for batch in tqdm(all_batch_samples):
        idx, result = process_batch(batch)
        output_thumbnail_preds.append(result)
        output_thumbnail_idx.append(idx)
        #pbar.update()
    output_thumbnail_idx = np.array(output_thumbnail_idx)
    output_thumbnail_preds = np.array(output_thumbnail_preds)
    output_thumbnail_preds = output_thumbnail_preds[output_thumbnail_idx]
    return output_thumbnail_preds

def get_tiles(batch_samples,
              patch_size=256,
              num_classes=2,
              convert_to_cat=True):
    """Generator function to yield image and mask tuples,

    Parameters
    ----------
    batch_samples: DataFrame
                   Subsetted dataframe as obtained from get_all_patches_from_slide
    patch_size: int
                Patch size
    convert_to_cat: bool
                    Should convert to categorical

    Returns
    -------
    X: tensor
    Y: tensor

    """
    images = []
    masks = []
    for _, batch_sample in batch_samples.iterrows():
        slide_contains_tumor = batch_sample['uid'].startswith('tumor')

        with WSIReader(batch_sample.slide_path, 40) as slide:
            tiles = DeepZoomGenerator(
                slide, tile_size=patch_size, overlap=0, limit_bounds=False)
            tile_loc = batch_sample.tile_loc  #[::-1]
            if isinstance(tile_loc, six.string_types):
                tile_row, tile_col = eval(tile_loc)
            else:
                tile_row, tile_col = tile_loc
            # the get_tile tuple required is (col, row)
            img = tiles.get_tile(tiles.level_count - 1, (tile_col, tile_row))
            (tile_x, tile_y), tile_level, _ = tiles.get_tile_coordinates(
                tiles.level_count - 1, (tile_col, tile_row))

        if slide_contains_tumor:
            json_filepath = batch_sample['json_filepath']
            polygons = get_annotation_polygons(json_filepath, 'shapely')
            mask = create_tumor_mask_from_tile(tile_x, tile_y, polygons,
                                               patch_size)
        else:
            mask = np.zeros((patch_size, patch_size))

        images.append(np.array(img))
        masks.append(mask)

    X_train = np.array(images)
    y_train = np.array(masks)
    if convert_to_cat:
        y_train = mask_to_categorical(y_train, num_classes, patch_size)
    return X_train, y_train





    

Populating the interactive namespace from numpy and matplotlib
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


/home/saket/anaconda3/lib/python3.5/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['imread']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [82]:
slide_path = '/Z/personal-folders/interns/saket/histopath_data/CAMELYON16/training/tumor/tumor_009.tif'
json_filepath = '/Z/personal-folders/interns/saket/histopath_data/CAMELYON16/training/lesion_annotations_json/tumor_009.json'
#all_samples = get_all_patches_from_slide(slide_path, json_filepath, False,
#                                         256)
#all_samples = all_samples.sample(frac=0.001)

all_samples = pd.read_table('/Z/personal-folders/interns/saket/github/pyvirchow/data/patch_df/validate_df_with_mask.tsv')
all_samples = all_samples.sample(frac=0.001)

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""




model = get_model()
model.load_weights('weights-improvement-12-0.98.hdf')


all_samples[['row', 'col']] = all_samples.tile_loc.astype(tuple).str.replace(' ','').str.replace(')', '').str.replace('(', '').str.split(',', expand=True)
#all_samples.uid.cat(x[:,0], sep='_')
all_samples['img_path1'] = all_samples[['uid', 'row', 'col']].apply(lambda x: '_'.join(x.values.tolist()),
                                                                    axis=1)

In [83]:
'x' + all_samples['img_path1']

21602    xnormal_159_276_153
20847     xtumor_046_267_164
10533     xtumor_046_314_177
43582       xtumor_097_66_91
26114     xtumor_046_258_126
50057     xtumor_011_407_170
24141     xtumor_069_296_187
39599     xtumor_085_223_257
34447     xtumor_085_180_138
9064     xnormal_130_237_198
23498     xtumor_085_201_245
26938     xtumor_011_443_279
9339      xtumor_046_275_203
34180     xtumor_065_396_188
16765     xtumor_031_649_222
49023     xtumor_046_279_177
52955     xtumor_085_188_266
32544     xtumor_011_435_313
19005     xtumor_046_276_196
6347     xnormal_097_522_123
2602       xtumor_097_96_448
56518     xtumor_085_181_289
18066    xnormal_097_538_296
14999     xtumor_046_202_186
37070     xtumor_011_439_242
25627      xtumor_005_668_98
44215     xtumor_069_399_302
10268      xtumor_085_97_154
7252     xnormal_092_345_153
51185     xtumor_046_385_172
32728     xtumor_085_210_260
9509     xnormal_092_297_229
29813     xtumor_046_249_172
52325     xtumor_011_617_249
860       xtum

In [72]:
x = pd.DataFrame(x)

In [73]:
x

,0,1
33801,263,210
35249,134,122
39456,324,225
30882,296,262
54382,654,246
56911,335,240
4793,448,216
31927,411,219
45847,405,274
56606,631,247


In [45]:
#all_samples.tile_loc.apply(pd.Series)
pd.DataFrame(all_samples.tile_loc.astype(tuple).values.tolist())

,0
0,"(263, 210)"
1,"(134, 122)"
2,"(324, 225)"
3,"(296, 262)"
4,"(654, 246)"
5,"(335, 240)"
6,"(448, 216)"
7,"(411, 219)"
8,"(405, 274)"
9,"(631, 247)"


In [40]:
#predicted_thumbnails = slide_level_map(slide_path=slide_path, batch_size=32, 
#                                       json_filepath=json_filepath, model=model)
def process_batch(args):
    idx, model, batch_samples, batch_size = args
    output_thumbnail_pred = None
    X, _ = get_tiles(batch_samples)
    X, _ = get_tiles_fast(batch_samples)

    if batch_samples.is_tissue.nunique(
    ) == 1 and batch_samples.iloc[0].is_tissue == False:
        # all patches in this row do not have tissue, skip them all
        output_thumbnail_pred = np.zeros(batch_size, dtype=np.float32)

    else:
        # make predictions
        preds = predict_on_batch(X, model)
        output_thumbnail_pred = preds.mean(axis=(1, 2))
    return idx, output_thumbnail_pred

In [41]:
%lprun -f process_batch process_batch((0, model, all_samples, len(all_samples.index)))


Timer unit: 1e-06 s

Total time: 8.24149 s
File: <ipython-input-40-1c5752ab191d>
Function: process_batch at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def process_batch(args):
     4         1          6.0      6.0      0.0      idx, model, batch_samples, batch_size = args
     5         1          3.0      3.0      0.0      output_thumbnail_pred = None
     6         1    7059359.0 7059359.0     85.7      X, _ = get_tiles(batch_samples)
     7         1     113903.0 113903.0      1.4      X, _ = get_tiles_fast(batch_samples)
     8                                           
     9         1        354.0    354.0      0.0      if batch_samples.is_tissue.nunique(
    10         1        609.0    609.0      0.0      ) == 1 and batch_samples.iloc[0].is_tissue == False:
    11                                                   # all patches in this row do not have tissue, skip them all
    12                       

In [33]:
len(all_samples.index)

324

In [28]:
%lprun -f slide_level_map slide_level_map(slide_path=slide_path, batch_size=32, json_filepath=json_filepath, model=model)



  7%|▋         | 7/102 [00:32<07:18,  4.62s/it]

*** KeyboardInterrupt exception caught in code being profiled.

Timer unit: 1e-06 s

Total time: 44.6272 s
File: <ipython-input-27-055c5ecb65c0>
Function: slide_level_map at line 317

Line #      Hits         Time  Per Hit   % Time  Line Contents
   317                                           def slide_level_map(model, slide_path, batch_size=32, json_filepath=None):
   318         1          3.0      3.0      0.0      all_samples = get_all_patches_from_slide(slide_path, json_filepath, False,
   319         1    1716894.0 1716894.0      3.8                                               256)
   320         1      46393.0  46393.0      0.1      all_samples = all_samples.sample(frac=0.01)
   321         1         22.0     22.0      0.0      n_samples = len(all_samples.index)
   322         1          1.0      1.0      0.0      all_batch_samples = []
   323       103        209.0      2.0      0.0      for idx, offset in enumerate(list(range(0, n_samples, batch_size))):
   324       102      41970.0    411.5      0.1          all_batch_samples.append(